In [6]:
from typing import List, Dict ,Tuple ,Any, Union
import numpy as np
from copy import deepcopy

class Solver:
    pass

"""
state:
    win_rounds
    1wins
    2wins
    1valids:[indices]
    2valids:[indices]
    mover:1 or 2 or 3
    1present
    2present
    winner
"""

def isterminal(state):
    rounds = state['win_rounds']
    if state['1wins'] >= rounds:
        return True
    elif state['2wins'] >= rounds:
        return True
    return False
def terminal_value(state):
    if state['1wins'] >= state['win_rounds']:
        return 1
    else:
        return 0

class GameNode:
    def __init__(self, state, parent = None):
        ###node_type = p1,p2, terminal
        #self.ntype = node_type
        self.regrets = []
        self.state = deepcopy(state)
        self.parent = parent
        self.childs = []
        self.value = 0.0
        self.prob_childs = []##抵达孩子的概率. 例如胜率.
        ###p1-p2节点的转移概率是我方的策略概率
        ###p2-p1节点的转移概率是对手的策略概率*胜率
    def expand(self):
        ##构建博弈树. 这一阶段不涉及策略概率问题, 仅构建所有可能的节点.
        ##返回一个bool表示这是否是终结状态
        s = self.state
        if isterminal(self.state):
            self.value = terminal_value(self.state)
            self.p2value = -self.value
            return False
        player = s['mover']
        if player == 3:
            ###结算节点
            s1 = s.copy()
            s1['1present'] = None
            s1['2present'] = None
            s1['mover'] = 1
            s2 = s1.copy()
            s1['1wins'] += 1
            s1['winner'] = 1
            s2['winner'] = 2
            s2['2wins'] += 2
            self.childs.append(GameNode(s1, self))
            self.childs.append(GameNode(s2, self))
        else:
            valids = s['1valids'] if player == 1 else s['2valids']
            for i, action in enumerate(valids):
                if player == 1:
                    n_state = s.copy()
                    n_state['1valids'] = valids[:i] + valids[i+1:]
                    n_state['1present'] = action
                    n_state['mover'] = 2
                    self.childs.append(GameNode(n_state, self))
                else:
                    n_state = s.copy()
                    n_state['2valids'] = valids[:i] + valids[i+1:]
                    n_state['2present'] = action
                    n_state['mover'] = 3
                    self.childs.append(GameNode(n_state, self))
        for child in self.childs:
            child.expand()
        return True
    def init_prob(self, bp_table):
        player = self.state['mover']
        if isterminal(self.state):
            return
        if player == 3:
            win_rate = bp_table[self.state['1present'],self.state['2present']]
            self.prob_childs.append(win_rate)
            self.prob_childs.append(1-win_rate)
            #print(self.prob_childs)
        else:
            valids = self.state['1valids'] if player == 1 else self.state['2valids']
            self.regrets = [0.001] * len(valids)
            self.prob_childs = [1/len(valids)] * len(valids)
        for child in self.childs:
            child.init_prob(bp_table)
    def update_values(self):
        if isterminal(self.state):
            return
        for child in self.childs:
            child.update_values()
        value = 0.0
        #print()
        #print(f"childs_length:{len(self.childs)}",f"prob_length:{len(self.prob_childs)}")
        #print(self.state)
        for i, child in enumerate(self.childs):
                #for j in range(len(child.childs)):
                #    value += self.prob_childs[i] * child.prob_childs[j] * child.childs[j].value
                value += self.prob_childs[i] * child.value
        p2value = 0.0
        for i, child in enumerate(self.childs):
            p2value += self.prob_childs[i] * child.p2value
        self.value = value
        self.p2value = p2value
    def update_regrets(self):
        if isterminal(self.state):
            return
        if self.state['mover'] == 1:
            for i in range(len(self.childs)):
                self.regrets[i] += max(self.childs[i].value - self.value,0)
        elif self.state['mover'] == 2:
            for i in range(len(self.childs)):
                self.regrets[i] += max(self.childs[i].p2value - self.p2value,0)
        for child in self.childs:
            child.update_regrets()
    def update_prob(self):
        ##必须先调用init_prob
        player = self.state['mover']
        if isterminal(self.state):
            return
        if player == 3:
            for child in self.childs:
                child.update_prob()
            return
        regrets = np.array(self.regrets)
        regrets[regrets < 0] = 0.0
        strategy = regrets/np.sum(regrets)
        valids = self.state['1valids'] if player == 1 else self.state['2valids']
        for i , action in enumerate(valids):
            self.prob_childs[i] = strategy[i]
        for child in self.childs:
            child.update_prob()
        return
class GameTree:
    def __init__(self, begin_state, bp_table):
        self.bp_table = bp_table
        self.root = GameNode(begin_state, None)
    def build(self):
        self.root.expand()
        self.root.init_prob(self.bp_table)
    def compute(self, iters):
        for _ in range(iters):
            self.root.update_values()
            self.root.update_regrets()
            self.root.update_prob()
        regs = np.array(self.root.regrets)
        #return regs
        regs[regs < 0] = 0.0
        return regs/np.sum(regs)
    

class SuperGameNode:
    def __init__(self, state, parent = None):
        ###node_type = p1,p2, terminal
        #self.ntype = node_type
        self.p1_regrets = []
        self.p2_regrets = []
        self.state = deepcopy(state)
        self.parent = parent
        self.childs = []
        p1_actions = state['1valids']
        p2_actions = state['2valids']
        self.actions = []
        for p1 in p1_actions:
            for p2 in p2_actions:
                self.actions.append((p1,p2))
        self.value = 0.0
        self.prob_childs = []##抵达孩子的概率. 例如胜率.
        ###p1-p2节点的转移概率是我方的策略概率
        ###p2-p1节点的转移概率是对手的策略概率*胜率
    def expand(self):
        ##构建博弈树. 这一阶段不涉及策略概率问题, 仅构建所有可能的节点.
        ##返回一个bool表示这是否是终结状态
        s = self.state
        if isterminal(self.state):
            self.value = terminal_value(self.state)
            return False
        stage = s['stage']
        if stage == 'account':
            ###结算节点
            s1 = s.copy()
            p1a, index1 = s['1present']
            p2a, index2 = s['2present']
            new_1valids = s['1valids'][:]
            new_1valids.insert(index1, p1a)
            new_2valids = s['2valids'][:]
            new_2valids.insert(index2, p2a)
            s1['1present'] = None
            s1['2present'] = None
            s1['stage'] = 'player'
            s2 = s1.copy()
            s1['2valids'] = new_2valids##输了放回
            s1['1wins'] += 1
            s1['winner'] = 1
            s2['1valids'] = new_1valids
            s2['winner'] = 2
            s2['2wins'] += 2
            self.childs.append(SuperGameNode(s1, self))
            self.childs.append(SuperGameNode(s2, self))
        else:
            p1_valids = s['1valids']
            p2_valids = s['2valids']
            for i, p1_action in enumerate(p1_valids):
                for j, p2_action in enumerate(p2_valids):
                    n_state = s.copy()
                    n_state['1valids'] = p1_valids[:i] + p1_valids[i+1:]
                    n_state['2valids'] = p2_valids[:j] + p2_valids[j+1:]
                    n_state['2present'] = (p2_action, i)
                    n_state['stage'] = 'account'
                    n_state['1present'] = (p1_action, j)
                    self.childs.append(SuperGameNode(n_state, self))
        for child in self.childs:
            child.expand()
        return True
    def init_prob(self, bp_table):
        stage = self.state['stage']
        if isterminal(self.state):
            return
        if stage == 'account':
            win_rate = bp_table[self.state['1present'][0],self.state['2present'][0]]
            self.prob_childs.append(win_rate)
            self.prob_childs.append(1-win_rate)
            #print(self.prob_childs)
        else:
            self.p1_regrets = [0.001] * len(self.state['1valids'])
            self.p2_regrets = [0.001] * len(self.state['2valids'])
            self.prob_childs = [1/len(self.actions)] * len(self.actions)
        for child in self.childs:
            child.init_prob(bp_table)
    def update_values(self):
        if isterminal(self.state):
            return
        for child in self.childs:
            child.update_values()
        value = 0.0
        #print()
        #print(f"childs_length:{len(self.childs)}",f"prob_length:{len(self.prob_childs)}")
        #print(self.state)
        p1strategy = self._compute_pi(1)
        p2strategy = self._compute_pi(2)
        for i in range(len(p1strategy)):
            for j in range(len(p2strategy)):
                self.prob_childs[i*len(p1strategy)+j] = p1strategy[i] * p2strategy[j]
        for i, child in enumerate(self.childs):
                #for j in range(len(child.childs)):
                #    value += self.prob_childs[i] * child.prob_childs[j] * child.childs[j].value
                value += self.prob_childs[i] * child.value
        p2value = 0.0
        self.value = value
    def _compute_pi(self, player):
        if player == 1:
            regs = np.array(self.p1_regrets)
            regs[regs < 0] = 0
            return regs/np.sum(regs)
        else:
            regs = np.array(self.p2_regrets)
            regs[regs < 0] = 0
            return regs/np.sum(regs)
    def update_regrets(self):
        if isterminal(self.state):
            return
        if self.state['stage'] == 'player':
            p1actions = self.state['1valids']
            p2actions = self.state['2valids']
            p1strategy = self._compute_pi(1)
            p2strategy = self._compute_pi(2)
            #print(self.value)
            #print('childs=', len(self.childs))
            #print(p1strategy)
            #print(self.state['stage'])
            for i, p1a in enumerate(p1actions):
                offsprings = self.childs[i*len(p2actions):(i+1)*len(p2actions)]
                benefit = 0.0
                for k, child in enumerate(offsprings):
                    benefit += p2strategy[k] * child.value
                self.p1_regrets[i] += max(benefit - self.value, 0)
            for i, p2a in enumerate(p2actions):
                offsprings = self.childs[i::len(p2actions)]
                benefit = 0.0
                for j, child in enumerate(offsprings):
                    benefit += p1strategy[j] * child.value
                self.p2_regrets[i] += max(self.value - benefit, 0)
        for child in self.childs:
            child.update_regrets()
    @property
    def probs(self):
        if self.state['stage']=='account':
            return None
        return self._compute_pi(1), self._compute_pi(2)
class SuperGameTree:
    def __init__(self, begin_state, bp_table):
        self.bp_table = bp_table
        self.root = SuperGameNode(begin_state, None)
    def build(self):
        self.root.expand()
        self.root.init_prob(self.bp_table)
    def compute(self, iters):
        print("initial probs")
        print(self.root.probs)
        for _ in range(iters):
            self.root.update_values()
            if _ == 0:
                print("intial value=", self.root.value)
            self.root.update_regrets()
            #self.root.update_prob()
        return self.root.probs
    
class SuperHelper(Solver):
    def __init__(self, bp_table, player1_decks, player2_decks):
        ##bp_table: 二维数组, 代表p1对p2的卡组的胜率.横轴p1,纵轴p2
        ##decks: list of deck names(string). 与横轴纵轴对应
        self.bp = bp_table
        self.p1_decks = player1_decks
        self.p2_decks = player2_decks
        total = len(bp_table)
        start_state = {
            'win_rounds':3,
            '1wins':0,
            '2wins':0,
            '1valids':[0,1,2,3],
            '2valids':[0,1,2,3],
            'stage':'player',
            '1present':None,
            '2present':None,
            'winner':0
        }
        self.tree = SuperGameTree(start_state, bp_table)
        self.tree.build()
        print("博弈树完成")
        #print(self.tree.root.childs)
    def best_strategy(self, iters = 10, ban_list = None):
        return self.tree.compute(iters)
class BPHelper(Solver):
    def __init__(self, bp_table, player1_decks, player2_decks):
        ##bp_table: 二维数组, 代表p1对p2的卡组的胜率.横轴p1,纵轴p2
        ##decks: list of deck names(string). 与横轴纵轴对应
        self.bp = bp_table
        self.p1_decks = player1_decks
        self.p2_decks = player2_decks
        #total = bp_table.shape[0]
        total = 4
        start_state = {
            'win_rounds':3,
            '1wins':0,
            '2wins':1,
            '1valids':[0,1,2,3],
            '2valids':[0,1,2,3],
            'mover':1,
            '1present':None,
            '2present':None,
            'winner':0
        }
        self.tree = GameTree(start_state, bp_table)
        self.tree.build()
        print("博弈树完成")
        #print(self.tree.root.childs)
    def best_strategy(self, iters = 10, ban_list = None):
        return self.tree.compute(iters)

In [7]:
bp_table = [[0.4,0.0,0.7,0.5],##水镜皇[水凝,水皇,双火,水牛]
           [0.4,0.0,0.6,0.5],##水刻
           [0.8,0.0,0.2,0.8],##双火
           [0.8,0.0,0.2,0.8]]##火刻
bp_table = np.array(bp_table)

rock = [[0.5, 0.0, 1.0],
       [1.0, 0.5, 0.0],
       [0.0, 1.0, 0.5]]
rock = np.array(rock)
#bp_table  = rock
helper = SuperHelper(bp_table, None,None)
helper.best_strategy(50)

博弈树完成
initial probs
(array([0.25, 0.25, 0.25, 0.25]), array([0.25, 0.25, 0.25, 0.25]))


IndexError: list assignment index out of range

In [13]:
now = helper.tree.root.childs[5].childs[1]

In [14]:
now.state

{'win_rounds': 3,
 '1wins': 0,
 '2wins': 2,
 '1valids': [0, 1, 2, 3],
 '2valids': [0, 1, 2, 3],
 'stage': 'player',
 '1present': None,
 '2present': None,
 'winner': 2}

In [16]:
rock_win = helper.tree.root.childs[1].childs[0].childs

In [17]:
[child.value for child in rock_win]

[0.5, 1.0, 0.0, 1.0, 0.0, 0.5]

1